### Combined the results of the 4 models and made a cumulative weighted prediction on the L&T tweets retrieved by scrapping L&T's twitter account

In [39]:
# Load previously trained LSTM Model

from keras.models import load_model

model_path ='lstm_model2.h5'
pretrained_lstm_model = load_model(model_path)
pretrained_cnn_model = load_model('cnn_model.h5')
pretrained_snn_model = load_model('snn_model.h5')

# # summarize model
# pretrained_lstm_model.summary()

In [40]:
# Load sample IMDb reviews csv, having ~6 movie reviews, along with their IMDb rating
import pandas as pd
sample_reviews = pd.read_csv("twitter_LT.csv")

sample_reviews.head(6)

,position,title,snippet,link,displayed_link,highlighs
0,0,Larsen & Toubro (@larsentoubro) / ...,The official Larsen & Toubro Corporate handle....,https://twitter.com/larsentoubro,https://twitter.com › larsentoubro,NaN
1,1,L&T Technology Services (LTTS) (@LnTTechservic...,We are a global leader in Engineering and R&D ...,https://twitter.com/LnTTechservices,https://twitter.com › LnTTechservices,NaN
2,2,Financial Express,@larsentoubro. ) announced that its Hydrocarbo...,https://twitter.com/FinancialXpress/status/176...,https://twitter.com › FinancialXpress › status,['larsentoubro']
3,3,Financial Express,@larsentoubro. ) announced that it has bagged ...,https://twitter.com/FinancialXpress/status/176...,https://twitter.com › FinancialXpress › status,['larsentoubro']
4,4,Financial Express,@larsentoubro. ) announced that its Hydrocarbo...,https://twitter.com/FinancialXpress/status/175...,https://twitter.com › FinancialXpress › status,['larsentoubro']
5,5,CNBC-TV18,@larsentoubro. ) slumps after news reports sug...,https://twitter.com/CNBCTV18News/status/175944...,https://twitter.com › status,['larsentoubro']


In [41]:
sample_reviews[sample_reviews['snippet'].isnull()]

,position,title,snippet,link,displayed_link,highlighs
8,8,"L&T Technology Services (LTTS) on X: ""Get read...",NaN,https://twitter.com/LnTTechservices/status/175...,https://twitter.com › LnTTechservices › status...,NaN


In [42]:
sample_reviews=sample_reviews[~sample_reviews['snippet'].isnull()]

In [43]:
unseen_reviews = sample_reviews['snippet']
from b2_preprocessing_function import CustomPreprocess
custom=CustomPreprocess()

# Assuming unseen_reviews is your DataFrame column with text data
unseen_processed = []

for review in unseen_reviews:
    # Check if review is a string, if not, convert it to a string
    if isinstance(review, str):
        review = custom.preprocess_text(review)
    else:
        review = str(review)  # Convert non-string values to string
        review = custom.preprocess_text(review)
    unseen_processed.append(review)


In [44]:
unseen_processed

['official larsen toubro corporate handle follow us latest news announcements upcoming events ',
 'global leader engineering er services visit https co nu q information engineeringthechange ',
 ' larsentoubro announced hydrocarbon vertical secured significant order ongc mhn tcpp pgc bgc project mtpbp ',
 ' larsentoubro announced bagged major contract ministry defence government india supply high ',
 ' larsentoubro announced hydrocarbon vertical secured onshore order offshore order prestigious client ',
 ' larsentoubro slumps news reports suggest saudi aramco defers bid process bn orders safaniyah oilfield projects ',
 'larsen toubro indian multinational engaged epc projects commissioned first indigenously manufactured electrolyser ',
 'cnbctv market shares larsen toubro larsentoubro fell fy revenue beat street estimates lower expected ',
 'every step larsentoubro brings mumbai spirit life lntseabridgemarathon readytorun stridesformumbai ',
 'larsen toubro commissions first indigenously

In [45]:
from keras_preprocessing.text import tokenizer_from_json

In [46]:
# Loading
import json
with open('b3_tokenizer.json') as f:
    data = json.load(f)
    loaded_tokenizer = tokenizer_from_json(data)

In [47]:
# Tokenising instance with earlier trained tokeniser
unseen_tokenized = loaded_tokenizer.texts_to_sequences(unseen_processed)

In [48]:
unseen_tokenized

[[1300, 65649, 5522, 1790, 144, 105, 1610, 321, 10018, 3738, 2535],
 [3023, 5073, 9534, 1507, 3432, 541, 371, 289, 3907, 2485, 3020],
 [4088, 15841, 16470, 10114, 841, 281528, 130123, 294985, 74398, 772],
 [4088, 20592, 1116, 2989, 9404, 18948, 4534, 2302, 5093, 471],
 [4088, 15841, 16470, 158690, 841, 24430, 841, 84296, 1774],
 [56482, 321, 2378, 3119, 9714, 31835, 5844, 2223, 6416, 3961, 128708, 2543],
 [65649, 3169, 289574, 7227, 51263, 2543, 42529, 88, 46138],
 [442058, 1237, 3002, 65649, 661, 9962, 4700, 771, 1064, 12331, 2947, 1860],
 [277, 1405, 2225, 6510, 2707, 91],
 [65649, 24835, 88, 46138, 134454],
 [2033, 9215, 14, 26329, 6302, 1545, 11516, 6559, 5333, 3070, 17354, 9978],
 [4088, 15841, 1802, 16470, 2354, 158690, 772, 178272, 18244, 3299],
 [2141, 785, 2309, 1142, 2468],
 [3453, 4379, 2378, 1207, 1642, 5005],
 [2154, 3961, 1091, 1756, 792, 17, 6730, 3955],
 [49967, 1600, 389499, 7004, 573, 467960, 317, 14, 5431, 25514],
 [1118, 45793, 84296, 772, 88774, 3071, 739, 3299],
 

In [49]:
# Pooling instance to have maxlength of 30 tokens
from keras.preprocessing.sequence import pad_sequences
unseen_padded = pad_sequences(unseen_tokenized, padding='post', maxlen=30)

In [50]:
unseen_padded

array([[  1300,  65649,   5522, ...,      0,      0,      0],
       [  3023,   5073,   9534, ...,      0,      0,      0],
       [  4088,  15841,  16470, ...,      0,      0,      0],
       ...,
       [134217,  20027,   7813, ...,      0,      0,      0],
       [  3503,   1827,   5638, ...,      0,      0,      0],
       [    10,   1373,   1642, ...,      0,      0,      0]], dtype=int32)

In [51]:
# Passing tokenised instance to the LSTM model for predictions
unseen_sentiments_snn = pretrained_snn_model.predict(unseen_padded)

unseen_sentiments_cnn = pretrained_cnn_model.predict(unseen_padded)

unseen_sentiments_lstm = pretrained_lstm_model.predict(unseen_padded)

4/4 [==============================] - 0s 9ms/step


2024-03-24 11:16:16.267668: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-24 11:16:16.382506: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-24 11:16:16.759010: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-24 11:16:16.888177: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-24 11:16:16.929242: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 1s 14ms/step


In [52]:
unseen_sentiments_lstm

array([[9.9999940e-01],
       [9.9973410e-01],
       [9.9992347e-01],
       [5.4984572e-05],
       [8.5752362e-01],
       [8.4617537e-01],
       [9.9928170e-01],
       [9.4250163e-09],
       [9.7778475e-01],
       [9.9909806e-01],
       [3.1686902e-01],
       [1.0811734e-07],
       [3.6157059e-04],
       [3.2758158e-01],
       [9.9979931e-01],
       [9.4731911e-07],
       [9.9424946e-01],
       [9.9999356e-01],
       [9.9999702e-01],
       [9.9998486e-01],
       [6.2555663e-02],
       [1.3292804e-02],
       [9.9999750e-01],
       [9.9997306e-01],
       [4.2312969e-02],
       [8.4397294e-07],
       [3.8027018e-03],
       [1.3786045e-07],
       [9.9999571e-01],
       [9.9999917e-01],
       [1.1259439e-04],
       [9.9999976e-01],
       [1.8068112e-02],
       [9.9983072e-01],
       [9.9999988e-01],
       [1.0000000e+00],
       [7.4797696e-01],
       [9.9550700e-01],
       [1.0000000e+00],
       [9.3890643e-01],
       [9.9503541e-01],
       [9.999831

In [54]:
# Writing model output file back to Google Drive
import numpy as np
sample_reviews['Predicted Sentiments SNN'] = np.round(unseen_sentiments_snn*10,1)
sample_reviews['Predicted Sentiments CNN'] = np.round(unseen_sentiments_cnn*10,1)
sample_reviews['Predicted Sentiments LSTM'] = np.round(unseen_sentiments_lstm*10,1)

df_prediction_sentiments_snn = pd.DataFrame(sample_reviews['Predicted Sentiments SNN'], columns = ['Predicted Sentiments SNN'])
df_prediction_sentiments_cnn = pd.DataFrame(sample_reviews['Predicted Sentiments CNN'], columns = ['Predicted Sentiments CNN'])
df_prediction_sentiments_lstm = pd.DataFrame(sample_reviews['Predicted Sentiments LSTM'], columns = ['Predicted Sentiments LSTM'])
df_tweet                 = pd.DataFrame(sample_reviews['snippet'], columns = ['snippet'])
# df_review_text           = pd.DataFrame(sample_reviews['Review Text'], columns = ['Review Text'])
# df_imdb_rating           = pd.DataFrame(sample_reviews['IMDb Rating'], columns = ['IMDb Rating'])


dfx=pd.concat([df_tweet, df_prediction_sentiments_snn,df_prediction_sentiments_cnn,df_prediction_sentiments_lstm], axis=1)

# dfx.to_csv("c2_IMDb_Unseen_Predictions.csv", sep=',', encoding='UTF-8')

dfx.head(50)

,snippet,Predicted Sentiments SNN,Predicted Sentiments CNN,Predicted Sentiments LSTM
0,The official Larsen & Toubro Corporate handle....,5.1,2.9,10.0
1,We are a global leader in Engineering and R&D ...,8.2,7.9,10.0
2,@larsentoubro. ) announced that its Hydrocarbo...,7.4,3.9,10.0
3,@larsentoubro. ) announced that it has bagged ...,2.8,7.3,0.0
4,@larsentoubro. ) announced that its Hydrocarbo...,8.8,9.2,8.6
5,@larsentoubro. ) slumps after news reports sug...,2.3,0.2,8.5
6,Larsen & Toubro (L&T) an Indian multinational ...,5.6,0.3,10.0
7,CNBCTV18Market | Shares of Larsen & Toubro (@l...,2.6,0.8,0.0
9,"With every step, @larsentoubro brings Mumbai's...",8.7,9.4,9.8
10,Larsen & Toubro (L&T) commissions first indige...,2.5,2.8,10.0


In [55]:
from happytransformer import HappyTextClassification

In [56]:
happy_tc=HappyTextClassification(model_type='DISTILBERT', model_name='distilbert-base-uncased-finetuned-sst-2-english',num_labels=2)

In [57]:
result=happy_tc.classify_text(dfx['snippet'][45])

In [58]:
result

TextClassificationResult(label='NEGATIVE', score=0.9897340536117554)

In [59]:
dfx['snippet'][96]

'@larsentoubro. and his team; L&T is doing several projects in Tanzania in water, power and other sectors. Image. India in Tanzania and 5 others.'

In [60]:
dfx=dfx.dropna(subset=['snippet'])
snippets=list(dfx['snippet'])
result=[]
for snippet in snippets:
    prediction=(happy_tc.classify_text(snippet))
    if prediction.label == 'POSITIVE':
        result.append(np.round((prediction.score)*10,1))
    else:
        result.append(np.round((1-prediction.score)*10,1))

In [61]:
result

[9.9,
 9.9,
 1.0,
 0.3,
 9.2,
 0.0,
 8.8,
 0.0,
 10.0,
 0.7,
 1.0,
 9.3,
 2.6,
 8.2,
 9.9,
 9.9,
 10.0,
 0.9,
 0.1,
 0.3,
 1.6,
 0.1,
 0.2,
 0.1,
 9.9,
 0.4,
 0.2,
 8.9,
 10.0,
 0.3,
 10.0,
 0.2,
 0.1,
 9.6,
 7.3,
 10.0,
 0.4,
 9.9,
 10.0,
 0.5,
 0.2,
 0.1,
 0.0,
 0.1,
 0.1,
 0.1,
 9.7,
 1.1,
 9.9,
 10.0,
 9.8,
 10.0,
 10.0,
 0.0,
 10.0,
 9.8,
 0.0,
 0.7,
 2.5,
 0.0,
 0.1,
 7.2,
 9.6,
 0.1,
 10.0,
 0.7,
 0.0,
 10.0,
 10.0,
 9.3,
 0.0,
 10.0,
 4.8,
 4.0,
 9.9,
 10.0,
 0.1,
 1.6,
 2.3,
 0.0,
 10.0,
 9.6,
 0.1,
 0.1,
 6.8,
 0.2,
 9.9,
 0.6,
 0.1,
 9.7,
 0.1,
 10.0,
 0.3,
 2.1,
 2.0,
 9.7,
 0.1,
 9.9,
 10.0]

In [62]:
df_prediction_sentiments_distilbert = pd.DataFrame(result, columns = ['Predicted Sentiments DistilBERT'])

In [63]:
dfx=pd.concat([dfx,df_prediction_sentiments_distilbert], axis=1)

In [64]:
dfx

,snippet,Predicted Sentiments SNN,Predicted Sentiments CNN,Predicted Sentiments LSTM,Predicted Sentiments DistilBERT
0,The official Larsen & Toubro Corporate handle....,5.1,2.9,10.0,9.9
1,We are a global leader in Engineering and R&D ...,8.2,7.9,10.0,9.9
2,@larsentoubro. ) announced that its Hydrocarbo...,7.4,3.9,10.0,1.0
3,@larsentoubro. ) announced that it has bagged ...,2.8,7.3,0.0,0.3
4,@larsentoubro. ) announced that its Hydrocarbo...,8.8,9.2,8.6,9.2
...,...,...,...,...,...
96,@larsentoubro. and his team; L&T is doing seve...,2.7,1.4,10.0,0.1
97,@RailMinIndia · @AshwiniVaishnaw · @MBPatil · ...,1.6,6.5,10.0,9.9
98,"@larsentoubro. at Gogamukh, Dhemaji. Built at ...",2.4,0.8,0.0,10.0
99,Love & Trust are key words for our relationshi...,9.3,9.7,10.0,NaN


In [65]:
import pandas as pd

# Assuming df is your DataFrame containing four numeric columns: col1, col2, col3, col4
# You can replace these with your actual column names
# You also need to define your custom weights for each column
custom_weights = {
    'Predicted Sentiments SNN': 0.1,
    'Predicted Sentiments CNN': 0.5,
    'Predicted Sentiments LSTM': 0.20,
    'Predicted Sentiments DistilBERT': 0.20
}

# Calculate the weighted average
def weighted_average(row):
    return np.round(sum(row[col] * custom_weights[col] for col in custom_weights.keys()))

# Create a new column 'weighted_avg' containing the weighted average
dfx['weighted_avg'] = dfx.apply(weighted_average, axis=1)

# Print the DataFrame to verify the changes
dfx


,snippet,Predicted Sentiments SNN,Predicted Sentiments CNN,Predicted Sentiments LSTM,Predicted Sentiments DistilBERT,weighted_avg
0,The official Larsen & Toubro Corporate handle....,5.1,2.9,10.0,9.9,6.0
1,We are a global leader in Engineering and R&D ...,8.2,7.9,10.0,9.9,9.0
2,@larsentoubro. ) announced that its Hydrocarbo...,7.4,3.9,10.0,1.0,5.0
3,@larsentoubro. ) announced that it has bagged ...,2.8,7.3,0.0,0.3,4.0
4,@larsentoubro. ) announced that its Hydrocarbo...,8.8,9.2,8.6,9.2,9.0
...,...,...,...,...,...,...
96,@larsentoubro. and his team; L&T is doing seve...,2.7,1.4,10.0,0.1,3.0
97,@RailMinIndia · @AshwiniVaishnaw · @MBPatil · ...,1.6,6.5,10.0,9.9,7.0
98,"@larsentoubro. at Gogamukh, Dhemaji. Built at ...",2.4,0.8,0.0,10.0,3.0
99,Love & Trust are key words for our relationshi...,9.3,9.7,10.0,NaN,NaN


In [66]:
dfx['overall-sentiment'] = np.where(dfx['weighted_avg'] >= 5, 4, 0)

In [67]:
dfx

,snippet,Predicted Sentiments SNN,Predicted Sentiments CNN,Predicted Sentiments LSTM,Predicted Sentiments DistilBERT,weighted_avg,overall-sentiment
0,The official Larsen & Toubro Corporate handle....,5.1,2.9,10.0,9.9,6.0,4
1,We are a global leader in Engineering and R&D ...,8.2,7.9,10.0,9.9,9.0,4
2,@larsentoubro. ) announced that its Hydrocarbo...,7.4,3.9,10.0,1.0,5.0,4
3,@larsentoubro. ) announced that it has bagged ...,2.8,7.3,0.0,0.3,4.0,0
4,@larsentoubro. ) announced that its Hydrocarbo...,8.8,9.2,8.6,9.2,9.0,4
...,...,...,...,...,...,...,...
96,@larsentoubro. and his team; L&T is doing seve...,2.7,1.4,10.0,0.1,3.0,0
97,@RailMinIndia · @AshwiniVaishnaw · @MBPatil · ...,1.6,6.5,10.0,9.9,7.0,4
98,"@larsentoubro. at Gogamukh, Dhemaji. Built at ...",2.4,0.8,0.0,10.0,3.0,0
99,Love & Trust are key words for our relationshi...,9.3,9.7,10.0,NaN,NaN,0


In [68]:
dfx.to_csv('predictions.csv',index=False)